In [1]:
# Import the necessary elements
from gaiaxpy import PhotometricSystem, load_additional_systems

In [2]:

PhotometricSystem.get_available_systems()

'DECam, Els_Custom_W09_S2, Euclid_VIS, Gaia_2, Gaia_DR3_Vega, Halpha_Custom_AB, H_Custom, Hipparcos_Tycho, HST_ACSWFC, HST_HUGS_Std, HST_WFC3UVIS, HST_WFPC2, IPHAS, JKC, JKC_Std, JPAS, JPLUS, JWST_NIRCAM, LSST, PanSTARRS1, PanSTARRS1_Std, Pristine, SDSS, SDSS_Std, Sky_Mapper, Stromgren, Stromgren_Std, WFIRST'

In [19]:
import os, shutil, numpy as np
from pathlib import Path

base_src = Path('/media/luis/Data/GC-Porto-Alegre/filters/SPLUS_gaiaxpy')  # tus .dat actuales
trial_root = Path('/media/luis/Data/GC-Porto-Alegre/filters/SPLUS_trials')
trial_root.mkdir(exist_ok=True)

# lee archivos fuente que terminan en .dat
src_files = sorted([f for f in base_src.iterdir() if f.suffix.lower() == '.dat'])
print("Archivos fuente encontrados:", [f.name for f in src_files])

# definimos las convenciones a probar
conventions = [
    # (subfolder_name, filename_transform_func, remove_header_bool)
    (None, lambda name: name, False),                                  # SPLUS.F378_v00001.dat en carpeta raíz
    ('SPLUS', lambda name: name.split('.',1)[1] if name.startswith('SPLUS.') else name, False),  # F378_v00001.dat en subcarpeta SPLUS/
    (None, lambda name: name.replace('.', '_', 1), False),            # SPLUS_F378_v00001.dat en raíz
    ('SPLUS', lambda name: name.replace('.', '_', 1).split('_',1)[1] if name.startswith('SPLUS_') else name, False), # F378_v00001.dat en subcarpeta con prefijo cambiado
    (None, lambda name: name.split('.',1)[1] if name.startswith('SPLUS.') else name, False),  # F378_v00001.dat en raíz (sin prefijo)
    # versiones sin header (por si la librería exige sólo números)
    (None, lambda name: name, True),
    ('SPLUS', lambda name: name.split('.',1)[1] if name.startswith('SPLUS.') else name, True),
]

# helper: copiar y (opcional) eliminar header
def prepare_trial(convention_idx):
    subfolder, name_fn, remove_header = conventions[convention_idx]
    trial_dir = trial_root / f"trial_{convention_idx:02d}"
    # limpiar si existe
    if trial_dir.exists():
        shutil.rmtree(trial_dir)
    trial_dir.mkdir(parents=True)
    if subfolder:
        (trial_dir / subfolder).mkdir()
    for f in src_files:
        newname = name_fn(f.name)
        dest_dir = trial_dir / subfolder if subfolder else trial_dir
        dest = dest_dir / newname
        # si remove_header True: copia limpiando la primera línea si empieza por letra o contiene comma/header
        if remove_header:
            # detect header: lee primeras 3 líneas y si hay non-numeric en la primera, lo quita
            with f.open('r') as fin:
                lines = fin.readlines()
            # detectar si primera línea contiene letras o comas -> eliminar
            first = lines[0].strip()
            try:
                # intentar parsear primeros tokens como floats
                float(first.split()[0])
                has_header = False
            except Exception:
                has_header = True
            if has_header:
                newlines = lines[1:]
            else:
                newlines = lines
            with dest.open('w') as fout:
                fout.writelines(newlines)
        else:
            shutil.copy(f, dest)
    return trial_dir

# función para intentar load_additional_systems en cada trial
def try_load(trial_dir):
    try:
        from gaiaxpy.generator.photometric_system import load_additional_systems, remove_additional_systems
        # eliminar config previa
        try:
            remove_additional_systems()
        except Exception:
            pass
        PhotometricSystem = load_additional_systems(str(trial_dir))
        # si no lanza excepción, devolvemos el enum o su representación
        return True, PhotometricSystem.get_available_systems()
    except Exception as e:
        return False, repr(e)

# ejecutar pruebas
results = []
for i in range(len(conventions)):
    print("\n=== Probando convención", i, "->", conventions[i], "=== ")
    td = prepare_trial(i)
    ok, info = try_load(td)
    results.append((i, td, ok, info))
    print("Trial dir:", td)
    print("Resultado OK?:", ok)
    print("Info:", info)
    if ok:
        print("\n¡ÉXITO con la convención", i, "-> carpeta:", td, "!")
        break

# resumen
print("\n--- Resumen de pruebas ---")
for i, td, ok, info in results:
    print(i, td.name, ok)
    if not ok:
        print("  error:", info)
    else:
        print("  systems:", info)


Archivos fuente encontrados: ['SPLUS.F378_v00001.dat', 'SPLUS.F395_v00001.dat', 'SPLUS.F410_v00001.dat', 'SPLUS.F430_v00001.dat', 'SPLUS.F515_v00001.dat', 'SPLUS.F660_v00001.dat', 'SPLUS.F861_v00001.dat', 'SPLUS.g_v00001.dat', 'SPLUS.i_v00001.dat', 'SPLUS.r_v00001.dat', 'SPLUS.z_v00001.dat']

=== Probando convención 0 -> (None, <function <lambda> at 0x7c1dc4a7dca0>, False) === 
No additional configuration exists.
Trial dir: /media/luis/Data/GC-Porto-Alegre/filters/SPLUS_trials/trial_00
Resultado OK?: False
Info: ValueError('No filter files found in the given directory. Please check your files.')

=== Probando convención 1 -> ('SPLUS', <function <lambda> at 0x7c1dc4a7d9d0>, False) === 
No additional configuration exists.
Trial dir: /media/luis/Data/GC-Porto-Alegre/filters/SPLUS_trials/trial_01
Resultado OK?: False
Info: ValueError('No filter files found in the given directory. Please check your files.')

=== Probando convención 2 -> (None, <function <lambda> at 0x7c1dc4904af0>, False) =